In [ ]:
# This file is part of FAST-OAD_CS23-HE : A framework for rapid Overall Aircraft Design of Hybrid
# Electric Aircraft.
# Copyright (C) 2022 ISAE-SUPAERO

# Import pyVPLM modules/functions
from pyvplm.core.definition import PositiveParameter, PositiveParameterSet
from pyvplm.addon.variablepowerlaw import buckingham_theorem

# Declare physical variables and set (limits have no impact since DOE is not constructed)
t = PositiveParameter("t", [0.01, 46172.16929583475], "N", "trust")
RHO = PositiveParameter("RHO", [1.225], "kg/m^3", "air density")
n_rot = PositiveParameter("n_rot", [16.66, 33.33], "Hz", "rotational speed")
D = PositiveParameter("D", [2.31], "m", "propeller diameter")
pitch = PositiveParameter("pitch", [0.087, 1.0472], "rad", "propeller pitch")
PITCH_REF = PositiveParameter("PITCH_REF", [0.087], "rad", "propeller reference pitch")
v = PositiveParameter("v", [5.0, 191.37333333333333], "m/s", "air speed")
parameter_set = PositiveParameterSet(t, RHO, n_rot, D, pitch, PITCH_REF, v)

# Define d and lambda_wind to be first elected as repetitive set
parameter_set.first("n_rot", "D", "RHO")
parameter_set.latex_render()

# Calculate pi parameters with first repetitive set found in parameters' order
pi_set, _ = buckingham_theorem(parameter_set, False)
pi_set.latex_render()

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.core.definition import Constraint, ConstraintSet
from pyvplm.addon.pixdoe import create_const_doe
from pyvplm.addon.variablepowerlaw import (
    declare_func_x_to_pi,
    reduce_parameter_set,
    declare_constraints,
)
from pyvplm.addon.comsoladdon import save_file

reduced_parameter_set, reduced_pi_set = reduce_parameter_set(
    parameter_set, pi_set, elected_output="t"
)
reduced_pi_set.latex_render()
reduced_parameter_set.latex_render()

constraint1 = Constraint("n_rot*D/2<=300.0")
parameters_constraints = declare_constraints(reduced_parameter_set, ConstraintSet(constraint1))

# Save function to translate x parameters to pi parameters
func_x_to_pi = declare_func_x_to_pi(reduced_parameter_set, reduced_pi_set)

results = create_const_doe(
    reduced_parameter_set,
    reduced_pi_set,
    func_x_to_pi,
    whished_size=300,
    parameters_constraints=parameters_constraints,
)
doeX = results[0]

In [ ]:
import numpy as np

n_rot_val = np.array(doeX[:, 0])
D_val = np.array(doeX[:, 1])
RHO_val = np.array(doeX[:, 2])
pitch_val = np.array(doeX[:, 3])
PITCH_REF_val = np.array(doeX[:, 4])
v_val = np.array(doeX[:, 5])

aoa_75 = pitch_val - np.arctan2(v_val * 2, 0.75 * n_rot_val * D_val * 2.0 * np.pi)

proper_index = np.where(abs(aoa_75 - 0.05) < 0.22)[0]

new_doeX = doeX[proper_index, :]

# Save .txt file compatible with COMSOL
file_name = "./data/doe_fixed_prop"
save_file(new_doeX, file_name, reduced_parameter_set, is_SI=True)

n_rot_val = np.array(new_doeX[:, 0])
D_val = np.array(new_doeX[:, 1])
RHO_val = np.array(new_doeX[:, 2])
pitch_val = np.array(new_doeX[:, 3])
PITCH_REF_val = np.array(new_doeX[:, 4])
v_val = np.array(new_doeX[:, 5])

In [ ]:
import os.path as pth
from fastga_he.models.propulsion.components.propulsor.propeller.components.performance_point import (
    ComputePropellerPointPerformance,
)
from fastga.command.api import generate_block_analysis

# Define relative path
DATA_FOLDER_PATH = "data"
DATA_FILE = pth.join(DATA_FOLDER_PATH, "input_tbm900.xml")

# Define the wing primary geometry parameters name as a list
var_inputs = [
    "data:geometry:propeller:average_rpm",
    "data:geometry:propeller:diameter",
    "data:aerodynamics:propeller:point_performance:rho",
    "data:aerodynamics:propeller:point_performance:twist_75",
    "data:aerodynamics:propeller:point_performance:twist_75_ref",
    "data:aerodynamics:propeller:point_performance:speed",
]

# Declare function
compute_propeller = generate_block_analysis(
    ComputePropellerPointPerformance(),
    var_inputs,
    str(DATA_FILE),
    overwrite=False,
)

In [ ]:
thrust_array = np.zeros_like(n_rot_val)
power_array = np.zeros_like(n_rot_val)
eta_array = np.zeros_like(n_rot_val)

for idx, n_rot_it in enumerate(n_rot_val):
    inputs_dict = {
        "data:geometry:propeller:average_rpm": (n_rot_it, "rps"),
        "data:geometry:propeller:diameter": (D_val[idx], "m"),
        "data:aerodynamics:propeller:point_performance:rho": (RHO_val[idx], "kg/m**3"),
        "data:aerodynamics:propeller:point_performance:twist_75": (pitch_val[idx], "rad"),
        "data:aerodynamics:propeller:point_performance:twist_75_ref": (PITCH_REF_val[idx], "rad"),
        "data:aerodynamics:propeller:point_performance:speed": (v_val[idx], "m/s"),
    }
    output_dict = compute_propeller(inputs_dict)
    thrust = output_dict["data:aerodynamics:propeller:point_performance:thrust"][0][0]
    power = output_dict["data:aerodynamics:propeller:point_performance:power"][0][0]
    eta = output_dict["data:aerodynamics:propeller:point_performance:efficiency"][0][0]
    thrust_array[idx] = thrust
    power_array[idx] = power
    eta_array[idx] = eta

proper_result_idx = np.where(abs(eta_array - 0.5) < 0.5)[0]

In [ ]:
thrust_array = thrust_array[proper_result_idx]
power_array = power_array[proper_result_idx]
eta_array = eta_array[proper_result_idx]

n_rot_val = n_rot_val[proper_result_idx]
D_val = D_val[proper_result_idx]
RHO_val = RHO_val[proper_result_idx]
pitch_val = pitch_val[proper_result_idx]
PITCH_REF_val = PITCH_REF_val[proper_result_idx]
v_val = v_val[proper_result_idx]

# Trying $\eta$ as a function of Ct and J

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

import pandas as pd

pi1 = eta_array
pi2 = thrust_array / (RHO_val * n_rot_val ** 2 * D_val ** 4)
pi3 = v_val / (n_rot_val * D_val)
doe_eta = np.c_[pi1, pi2, pi3]
doe_eta = pd.DataFrame(doe_eta, columns=["pi1", "pi2", "pi3"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_eta.values, elected_pi0="pi1", order=3, log_space=True, ymax_axis=10, test_mode=True
)
perform_regression(doe_eta.values, model, choosen_model=10)

# Comparison with BEMT computation

In [ ]:
import fastoad.api as oad
import fastga.utils.postprocessing.post_processing_api as api_plots

DATA_FILE_PROP = pth.join(DATA_FOLDER_PATH, "perf_prop_tbm900.xml")
reference_aircraft = oad.DataFile(DATA_FILE_PROP)

thrust = reference_aircraft["data:aerodynamics:propeller:sea_level:thrust"].value
speed_tbm900 = reference_aircraft["data:aerodynamics:propeller:sea_level:speed"].value
efficiency = reference_aircraft["data:aerodynamics:propeller:sea_level:efficiency"].value

fig = api_plots.propeller_efficiency_map_plot(DATA_FILE_PROP, sea_level=True)
fig.show()

In [ ]:
from scipy.interpolate import interp2d

effy_func = interp2d(thrust, speed_tbm900, efficiency)

In [ ]:
def effy_with_vplm(thrust, speed, rot_speed, reference_aircraft):

    d_tbm900 = reference_aircraft["data:geometry:propeller:diameter"].value[0]
    rho_tbm900 = 1.225

    ct = thrust / (rho_tbm900 * rot_speed ** 2 * d_tbm900 ** 4)
    j = speed / (rot_speed * d_tbm900)

    effy = (
        10 ** -0.39665
        * ct
        ** (
            0.04718 * np.log10(j) ** 3
            + 0.00643 * np.log10(ct) ** 2
            - 0.26803 * np.log10(ct)
            + 0.82513 * np.log10(j)
            + 0.27029 * np.log10(ct) * np.log10(j)
            - 0.58403
        )
        * j ** (0.74918 - 0.34468 * np.log10(j) - 0.03549 * np.log10(j) ** 2)
    )
        
    return effy

In [ ]:
# Define relative path
DATA_FOLDER_PATH = "data"
DATA_FILE_PROP = pth.join(DATA_FOLDER_PATH, "perf_prop_tbm900.xml")
tbm900 = oad.DataFile(DATA_FILE_PROP)

n_rot_tbm900 = tbm900["data:geometry:propeller:average_rpm"].value[0] / 60
n_rot_array = np.full_like(thrust, n_rot_tbm900)

thrust_mesh, speed_tbm900_mesh = np.meshgrid(thrust, speed_tbm900)
effy_vplm = effy_with_vplm(thrust_mesh, speed_tbm900_mesh, n_rot_array, tbm900)

In [ ]:
import plotly.graph_objects as go

fig_2 = go.Figure()
prop_scatter = go.Contour(
    x=thrust,
    y=speed_tbm900,
    z=effy_vplm,
    line_smoothing=0.85,
    ncontours=40,
    zmax=np.max(effy_vplm),
    zmin=0.0,
    colorscale="RdBu",
)
fig_2.add_trace(prop_scatter)

fig_2.update_layout(
    title_text="Propeller efficiency map at sea level with VPLM",
    title_x=0.5,
    yaxis_title="True airspeed [m/s]",
    xaxis_title="Thrust [N]",
    font_size=15,
)

fig_2 = go.FigureWidget(fig_2)
fig_2.show()

In [ ]:
rel_diff_in_eff = (effy_vplm - efficiency) / efficiency * 100

fig_3 = go.Figure()
prop_scatter = go.Contour(
    x=thrust,
    y=speed_tbm900,
    z=rel_diff_in_eff,
    line_smoothing=0.85,
    ncontours=40,
    zmax=np.max(rel_diff_in_eff),
    zmin=0.0,
    colorscale="RdBu",
)
fig_3.add_trace(prop_scatter)

fig_3.update_layout(
    title_text="Propeller efficiency difference",
    title_x=0.5,
    yaxis_title="True airspeed [m/s]",
    xaxis_title="Thrust [N]",
    font_size=15,
)

fig_3 = go.FigureWidget(fig_3)
fig_3.show()